In [ ]:
import shutil
outdir = './output'
shutil.rmtree(outdir)

**Improve your business vision**

In [ ]:
vision = """

"""

additional_prompt = ""

In [ ]:
import os
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
import pandas as pd
from importlib import reload

import lib.refine_vision as this
reload(this)

res = this.chain.invoke({"vision": vision, "additional_prompt":additional_prompt})
vision = {"vision": res.content.replace("\n", "")}
pd.DataFrame([vision]).to_csv('./output/vision.csv',index=False)

In [ ]:
print(res.content)

**Discovery of Main Jobs and Job Performers**

In [ ]:
additional_prompt = ""

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.discover_performers_and_jobs as this
reload(this)

res = this.chain.invoke({"vision": vision, "additional_prompt": additional_prompt})
res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])['performers_and_jobs_list']
performers_and_jobs = pd.DataFrame(res_dict)

performers_and_jobs.to_csv('./output/performers_and_jobs.csv',index=False)

performers_and_jobs

**Rate the different possible markets (aka Job_Performers + jobs) using Disciplined Enterpreneur book**

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.score_performers_and_jobs as this
reload(this)

scores = list()
import numpy as np
list_df = np.array_split(performers_and_jobs, 5)

for dataframe in list_df:  
    res = this.chain.invoke({"vision": vision, "performers_and_jobs": dataframe, "additional_prompt": additional_prompt})
    res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])["score_performers_and_jobs"]
    score = pd.DataFrame(res_dict)
    scores.append(score)

performers_score = pd.concat(scores)


columns_with_scores = performers_score.filter(like='Score', axis=1)
total_score_column = pd.Series(columns_with_scores.sum(axis=1), name='Total_Score')
performers_score.insert(1, 'Total_Score', total_score_column)
performers_score = performers_score.sort_values(by='Total_Score', ascending=False)

performers_score.to_csv('./output/performers_score.csv',index=False)


performers_score


**Synthesis of Job Performers**

In [ ]:
selected_job_performers = "Educators"
selected_main_job = "Create an engaging curriculum"
number = "2"
additional_prompt   = ""

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.generate_performers as this
reload(this)

res = this.chain.invoke({"number": number, "job_performers": selected_job_performers, "additional_prompt": additional_prompt})

performers = pd.DataFrame(json.loads(res.content)["job_performers"])
performers.to_csv('./output/performers.csv',index=False)

performers

**Automated Interviews**

In [ ]:
import json

from importlib import reload

import lib.interview_performers as this
reload(this)

interviews = []
for index, row in performers.iterrows():
    performer = row.to_json()

    main_job_related = f"""Answer the following question focusing and thinking about the main job {selected_main_job} applied to what you are doing as {row['Profession']}.
    If the main job {selected_main_job} is not related to you, simply answer \"This is not my main job\" and skip the question."""

    # main_job_related = ""

    res = this.chain.invoke({"job_performer": performer, "main_job_related": main_job_related})
    res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])
    
    interview = pd.DataFrame(res_dict["interview"])
    
    job_performer_dict = row.to_dict()

    job_performer_dict.update({"interview": interview})

    interviews.append(job_performer_dict)

In [ ]:
import os
outdir = './output/interviews'
if not os.path.exists(outdir):
    os.mkdir(outdir)

for interview in interviews:
    interview["interview"].to_csv(
        os.path.join(outdir,"_".join((
        interview["Name"].replace(" ", ""),
        interview["Profession"].replace(" ", ""),
        interview["Industry"].replace(" ", ""),
        interview["Experience_level"].replace(" ", ""),
        ))+".csv"),
        index=False)

**Analyze Interviews**

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.analyse_interviews as this
reload(this)

analysis_list = []
for interview in interviews:
    res = this.chain.invoke({"interview":interview})
    res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])["analysis"]
    analysis_list.append(pd.DataFrame(res_dict))

analysis = pd.concat(analysis_list)
analysis.to_csv('./output/analysis.csv',index=False)

In [ ]:
analysis = pd.read_csv('./output/analysis.csv')

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.categorize_job_steps as this
reload(this)

job_steps = analysis["Job Step"]

res = this.chain.invoke({"main_job":selected_main_job, "job_performers": selected_job_performers, "job_steps":job_steps})
res_dict = (json.loads(res.additional_kwargs['function_call']['arguments']))

job_steps_categorized = pd.DataFrame(res_dict['analysis'])



In [ ]:
job_steps_categorized


In [ ]:
analysis = pd.merge(analysis, job_steps_categorized[["Job Step", "Category"]], on='Job Step', how='outer')

In [ ]:
analysis

In [ ]:
custom_order = [
    'Define',
    'Locate',
    'Prepare',
    'Confirm',
    'Execute',
    'Monitor',
    'Modify',
    'Conclude'
]


# Create a Categorical data type with the custom order
category_dtype = pd.CategoricalDtype(categories=custom_order, ordered=True)

# Apply the custom order to the 'Job Step' column
analysis['Job Step'] = analysis['Job Step'].astype(category_dtype)

# Sort the DataFrame based on the 'Job Step' column
df = analysis.sort_values(by='Job Step')

df

In [ ]:
analysis['Job Step']